# Final parcellation single subject pipeline 

In [ ]:
from pathlib import Path
import numpy as np
import nibabel as nib 


from smoothing import smooth_surface_graph
from gradient import compute_gradients, compute_gradient_average, build_mesh_graph, \
                     save_gradient_mgh, load_gradient_mgh
from watershed import watershed_by_flooding, save_labels_mgh \
                    , load_labels_mgh
from visualization import visualize_brain_surface

In [ ]:
# Path LOCATIONS
config = {
"fsavg6_dir": Path(r"D:\Data_Conn_Preproc\fsaverage6"),
"subjects_dir": Path(r"D:\Data_Conn_Preproc\PPSFACE_N20")
}

subject = f"{1:02d}"
hemisphere = 'lh'
run = 1

subj_dir = config["subjects_dir"] / f"sub-{subject}"
# Create output directory if it doesn't exist.
(subj_dir / "outputs_surface").mkdir(exist_ok=True, parents=True)
# Define paths using pathlib
surf_fmri_path = subj_dir / "func" / f"surf_conn_sub{subject}_run{run}_{hemisphere}.func.fsaverage6.mgh"
surface_path = config["fsavg6_dir"] / "surf" / f"{hemisphere}.white"
surface_inf_path = config["fsavg6_dir"] / "surf" / f"{hemisphere}.inflated"

vol_fmri_file = subj_dir / "func" / f"niftiDATA_Subject{subject}_Condition000_run{run}.nii.gz"
brain_mask_path = subj_dir / f"sub{subject}_freesurfer" / "mri" / "brainmask.mgz"


# vol_fmri_file = r"D:\DATA_min_preproc\dataset_PPSFace2\sub-01\func\wsraAS_BO_PPS_S01_005_Rest.nii"
# brain_mask_path = r"D:\DATA_min_preproc\dataset_PPSFace2\sub-01\sub01_freesurfer\mri\brainmask.mgz"


# To test
gm_mask_path = subj_dir / f"sub{subject}_freesurfer" / "mri" / f"{hemisphere}.ribbon.mgz"

output_gradient_dir = subj_dir / "outputs_surface" / "gradient"
output_labels_dir = subj_dir / "outputs_surface" / "labels"

print('all paths defined')

In [ ]:
coords, faces = nib.freesurfer.read_geometry(str(surface_path))
coords_, faces_ = nib.freesurfer.read_geometry(str(surface_inf_path))
graph = build_mesh_graph(faces)


## PCA on the volumne data

In [ ]:
import nibabel as nib
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from nilearn import plotting
from nilearn.image import resample_img


# # Min preproc data
# vol_fmri_file = r"D:\DATA_min_preproc\dataset_PPSFace2\sub-01\func\wsraAS_BO_PPS_S01_005_Rest.nii"
# brain_mask_path = r"D:\DATA_min_preproc\dataset_PPSFace2\sub-01\sub01_freesurfer\mri\brainmask.mgz"

# # Conn preproc data
vol_fmri_file = r"D:\Data_Conn_Preproc\PPSFACE_N20\sub-01\func\niftiDATA_Subject01_Condition000_run1.nii.gz"
brain_mask_path = r"D:\Data_Conn_Preproc\PPSFACE_N20\sub-01\sub01_freesurfer\mri\brainmask.mgz"


# Load the images using nibabel
fmri_img = nib.load(str(vol_fmri_file))
mask_img = nib.load(brain_mask_path)


# resample the mask to the right one
mask_img = resample_img(
    mask_img,
    target_affine=fmri_img.affine,
    target_shape=fmri_img.get_fdata().shape[:-1],
    interpolation='nearest',
    force_resample=True
)

fmri_data = fmri_img.get_fdata()
mask_data = mask_img.get_fdata().astype(bool)  # Convert mask to boolean



# 1. Load the fMRI data and brain mask in right shape
X = fmri_data[mask_data].T  # Shape: (nt, n_voxels)

# print shape 
print(X.shape)
# print mean and var with their shape
print('mean', X.mean(axis=0), 'shape', X.mean(axis=0).shape)
print('var', X.var(axis=0), 'shape', X.var(axis=0).shape)

# normalize the data
X = (X - X.mean(axis=0)) / X.std(axis=0) 
X = np.nan_to_num(X)

# print mean and var with their shape
print('after mean', X.mean(axis=0), 'shape', X.mean(axis=0).shape)
print('after var', X.var(axis=0), 'shape', X.var(axis=0).shape)
# 4. Run PCA on the time series data
n_components = 20  # Number of PCA components you want to extract
pca = PCA(n_components=n_components)
temporal_modes = pca.fit_transform(X)

# The PCA components (spatial modes) are stored in pca.components_
# Shape of components: (n_components, n_voxels)
spatial_modes = pca.components_

# 5. Put the spatial modes back into a volumetric (3D) format.
#    Create an empty volume with the same shape as the brain mask for each component.
nx, ny, nz = mask_data.shape
spatial_modes_volumes = []

for i in range(n_components):
    vol = np.zeros((nx, ny, nz))
    vol[mask_data] = spatial_modes[i]
    spatial_modes_volumes.append(vol)

# 6. Plot the spatial modes.
# Use the affine from the original fMRI data for correct spatial orientation.
affine = fmri_img.affine

# Plot each PCA component using Nilearn's plot_stat_map.
# The `display_mode='ortho'` shows axial, sagittal, and coronal views.
for i, vol in enumerate(spatial_modes_volumes):
    # Create a Nifti image for the PCA component
    component_img = nib.Nifti1Image(vol, affine)
    
    # Plot the component with a symmetric colormap to visualize both positive and negative contributions.
    display = plotting.plot_stat_map(
        component_img,
        title=f'PCA Component {i+1}',
        display_mode='ortho',  # or use 'z' for axial slices montage, etc.
        cut_coords=(0, 0, 0),   # Adjust cut_coords or let Nilearn choose automatically
        cmap='cold_hot',        # symmetric colormap
        colorbar=True,
        threshold=np.percentile(np.abs(vol[mask_data]), 90) * 0.5  # optional thresholding for clarity
    )
    

print('explained variance:', pca.explained_variance_ratio_)
print('explained variance sum:', pca.explained_variance_ratio_.sum())

In [ ]:
print(X.var(axis=0).sum())
print(X.shape)

In [ ]:
import nibabel as nib
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from nilearn import plotting
from nilearn.image import resample_img


# # Min preproc data
# vol_fmri_file = r"D:\DATA_min_preproc\dataset_PPSFace2\sub-01\func\wsraAS_BO_PPS_S01_005_Rest.nii"
# brain_mask_path = r"D:\DATA_min_preproc\dataset_PPSFace2\sub-01\sub01_freesurfer\mri\brainmask.mgz"

# # Conn preproc data
vol_fmri_file = r"D:\Data_Conn_Preproc\PPSFACE_N20\sub-01\func\niftiDATA_Subject01_Condition000_run1.nii.gz"
brain_mask_path = r"D:\Data_Conn_Preproc\PPSFACE_N20\sub-01\sub01_freesurfer\mri\brainmask.mgz"


# Load the images using nibabel
fmri_img = nib.load(str(vol_fmri_file))
mask_img = nib.load(brain_mask_path)


# resample the mask to the right one
mask_img = resample_img(
    mask_img,
    target_affine=fmri_img.affine,
    target_shape=fmri_img.get_fdata().shape[:-1],
    interpolation='nearest',
    force_resample=True
)

fmri_data = fmri_img.get_fdata()
mask_data = mask_img.get_fdata().astype(bool)  # Convert mask to boolean



# 1. Load the fMRI data and brain mask in right shape
X = fmri_data[mask_data].T  # Shape: (nt, n_voxels)



# 4. Run PCA on the time series data
n_components = 17  # Number of PCA components you want to extract
pca = PCA(n_components=n_components)
temporal_modes = pca.fit_transform(X)

# The PCA components (spatial modes) are stored in pca.components_
# Shape of components: (n_components, n_voxels)
spatial_modes = pca.components_

# 5. Put the spatial modes back into a volumetric (3D) format.
#    Create an empty volume with the same shape as the brain mask for each component.
nx, ny, nz = mask_data.shape
spatial_modes_volumes = []

for i in range(n_components):
    vol = np.zeros((nx, ny, nz))
    vol[mask_data] = spatial_modes[i]
    spatial_modes_volumes.append(vol)

# 6. Plot the spatial modes.
# Use the affine from the original fMRI data for correct spatial orientation.
affine = fmri_img.affine

# Plot each PCA component using Nilearn's plot_stat_map.
# The `display_mode='ortho'` shows axial, sagittal, and coronal views.
for i, vol in enumerate(spatial_modes_volumes):
    # Create a Nifti image for the PCA component
    component_img = nib.Nifti1Image(vol, affine)
    
    # Plot the component with a symmetric colormap to visualize both positive and negative contributions.
    display = plotting.plot_stat_map(
        component_img,
        title=f'PCA Component {i+1}',
        display_mode='ortho',  # or use 'z' for axial slices montage, etc.
        cut_coords=(0, 0, 0),   # Adjust cut_coords or let Nilearn choose automatically
        cmap='cold_hot',        # symmetric colormap
        colorbar=True,
        threshold=np.percentile(np.abs(vol[mask_data]), 90) * 0.5  # optional thresholding for clarity
    )
    

print('explained variance:', pca.explained_variance_ratio_)
print('explained variance sum:', pca.explained_variance_ratio_.sum())

# ICA on volumne data

In [ ]:
import nibabel as nib
import numpy as np
from sklearn.decomposition import FastICA
import matplotlib.pyplot as plt
from nilearn import plotting
from nilearn.image import resample_img

# =============================================================================
# File paths (choose the appropriate preprocessed data)
# # For Conn preproc data:
# vol_fmri_file = r"D:\Data_Conn_Preproc\PPSFACE_N20\sub-01\func\niftiDATA_Subject01_Condition000_run1.nii.gz"
# brain_mask_path = r"D:\Data_Conn_Preproc\PPSFACE_N20\sub-01\sub01_freesurfer\mri\brainmask.mgz"

# =============================================================================
# Load the images using nibabel
fmri_img = nib.load(vol_fmri_file)
mask_img = nib.load(brain_mask_path)

# Resample the mask to match the fMRI data
mask_img = resample_img(
    mask_img,
    target_affine=fmri_img.affine,
    target_shape=fmri_img.get_fdata().shape[:-1],
    interpolation='nearest',
    force_resample=True
)

fmri_data = fmri_img.get_fdata()  # Expected shape: (nx, ny, nz, nt)
mask_data = mask_img.get_fdata().astype(bool)  # Convert to boolean mask

# =============================================================================
# Extract the time series from brain voxels
# Reshape the data to have shape: (n_timepoints, n_voxels)
X = fmri_data[mask_data].T

# =============================================================================
# Set the number of sources and perform ICA
n_components = 7 # This is where you set the number of ICA sources
ica = FastICA(n_components=n_components, random_state=0)
temporal_modes = ica.fit_transform(X)   # S has shape (380, n_components)

# The ICA components (spatial maps) are stored in ica.components_
# Each row corresponds to one spatial source (shape: (n_voxels,))
spatial_modes = ica.components_

# =============================================================================
# Reconstruct the spatial maps back into 3D volumes
nx, ny, nz = mask_data.shape
ica_maps = []

for i in range(n_components):
    vol = np.zeros((nx, ny, nz))
    vol[mask_data] = spatial_modes[i]
    ica_maps.append(vol)

# =============================================================================
# Plot the ICA spatial sources using Nilearn's plot_stat_map
affine = fmri_img.affine

for i, vol in enumerate(ica_maps):
    # Create a Nifti image for each ICA source
    source_img = nib.Nifti1Image(vol, affine)
    
    # Plot the source using orthogonal slices (axial, sagittal, coronal)
    display = plotting.plot_stat_map(
        source_img,
        title=f'ICA Source {i+1}',
        display_mode='ortho',  # Change to 'z', 'x', or 'y' for other views
        cut_coords=(0, 0, 0),   # You can adjust these coordinates or omit to auto-select
        cmap='cold_hot',        # A symmetric colormap to show positive/negative contributions
        colorbar=True,
        threshold=np.percentile(np.abs(vol[mask_data]), 90) * 0.5  # Optional thresholding
    )

# Display all plots
plotting.show()

In [ ]:
print('temporal_modes:', temporal_modes.shape)

In [ ]:
# time = np.arange(temporal_modes.shape[0])
# plt.figure(figsize=(12, 8))
# for i in range(temporal_modes.shape[1]):
#     plt.plot(time, temporal_modes[:, i], label=f'IC {i+1}')
# plt.xlabel("Time (TR)")
# plt.ylabel("Amplitude")
# plt.title("Independent Component Time Series")
# plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small')
# plt.tight_layout()
# plt.show()

# Parcellation with ICA or PCA signals

In [ ]:
# Normalize the surf fmri data
surf_fmri = nib.load(str(surf_fmri_path)).get_fdata().squeeze()
surf_fmri_n = (surf_fmri - np.mean(surf_fmri, axis=1, keepdims=True)) / np.std(surf_fmri, axis=1, keepdims=True)

# compute the correlation matrix (for normalized data, the correlation matrix is the same as the covariance matrix)
corr_matrix = (surf_fmri_n @ temporal_modes)  / (surf_fmri_n.shape[1] - 1)

# make the similarty matrix as the correlation of the correlation matrix
sim_matrix = np.corrcoef(corr_matrix)
del corr_matrix

print('shape sim matrix:',sim_matrix.shape)

In [ ]:
visualize_brain_surface(coords_, faces_, sim_matrix[:,1])

In [ ]:
sim_matrix_smoothed = smooth_surface_graph(graph, sim_matrix, iterations=5)

In [ ]:
gradients = compute_gradients(graph, sim_matrix_smoothed)
gradients_sum = gradients.sum(axis=1)
gradient_smoothed = smooth_surface_graph(graph, gradients_sum, iterations=10)

In [ ]:
visualize_brain_surface(coords_, faces_, gradient_smoothed)

In [ ]:
labels = watershed_by_flooding(graph, gradient_smoothed)

In [ ]:
visualize_brain_surface(coords_, faces_, labels)